<h1>Создание витрины данных базового уровня энергопотребления на объекте</h1>

In [1]:
import numpy as np
import pandas as pd
import pickle
import os
from dotenv import load_dotenv
from data_wrapper import db_wrapper
from clickhouse_driver import Client
import datetime as dt

In [2]:
# Период мониторинга для получения данных, на которых будет проводиться обучение модели
start_date = input('Введите начало периода в формате YYYY-MM-DD:')
end_date = input('Введите конец периода в формате YYYY-MM-DD:')
startYear, startMonth, startDay = map(int, start_date.split('-'))
endYear, endMonth, endDay = map(int, end_date.split('-'))

# <h1>Расчет базового уровня потребления</h1>

In [3]:
load_dotenv()

HOST = os.getenv('BASE_HOST')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('BASE_PASSWORD')
CA = os.getenv('CA')
READONLY = os.getenv('BASE_READONLY')

wrap = db_wrapper.ClickHouseWrapper(host=HOST, user=USERNAME, password=PASSWORD, ca=CA)
client = Client(host=HOST, user=USERNAME, password=PASSWORD, ca_certs=CA, secure=True)

In [4]:
sensors = [
# Потребление компрессоров
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total AP energy", 
    "channel": 2, "phase": 0, "name_in_df": "compressor1", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total AP energy",
    "channel": 3, "phase": 0, "name_in_df": "compressor2", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total AP energy",
    "channel": 4, "phase": 0, "name_in_df": "compressor3", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_145", "measurement": "Total AP energy",
    "channel": 1, "phase": 0, "name_in_df": "compressor4", "mode": "max-min"
},
# Внешняя температура
{
    "db": "genesis_arena", "name": "weather_owm", "measurement": "Temperature",
    "channel": 0, "phase": 0, "name_in_df": "temp_outside", "mode": "mean"
}
]

start = dt.datetime(startYear, startMonth, startDay, 0, 0, 0) - dt.timedelta(hours=3)
end = dt.datetime(endYear, endMonth, endDay, 0, 0, 0) - dt.timedelta(hours=3)
presample = dt.timedelta(days=1)

data = wrap.get_particular_sensors(start, end, sensors, presample_time=presample,  without_confidence=True)

In [5]:
data = (data
 .tz_localize(None)
 .reset_index(names='time')
 .assign(compressors=lambda df: df[['compressor1', 'compressor2', 'compressor3', 'compressor4']].sum(axis=1))
 .pipe(lambda df: df.drop(columns=['compressor1', 'compressor2', 'compressor3', 'compressor4']))
 )

<h3>Температура льда</h3>

In [6]:
sensors = [
# Температура льда
{
    "db": "genesis_arena", "name": "msw-v3_2", "measurement": "Temperature", 
    "channel": 0, "phase": 0, "name_in_df": "temp_ice", "mode": "mean"
}
]

start = dt.datetime(startYear, startMonth, startDay, 0, 0, 0) - dt.timedelta(hours=3)
end = dt.datetime(endYear, endMonth, endDay, 0, 0, 0) - dt.timedelta(hours=3)
presample = dt.timedelta(minutes=1)

ice_df = wrap.get_particular_sensors(start, end, sensors, presample_time=presample,  without_confidence=True)
ice_df = ice_df.tz_localize(None).reset_index(names='time')

In [7]:
ice_df = (ice_df
 # Обработка аномальных значений
 .assign(temp_ice=lambda df: np.where(df['temp_ice'].between(-10, 3), df['temp_ice'], np.NaN))
 .assign(temp_ice=lambda df: df['temp_ice'].ffill())
 # Дифференцируем, экспоненциально сглаженный, ряд для нахождения заливки
 .assign(diff=lambda df: df['temp_ice'].ewm(span=60).mean().diff())
 .assign(flood=lambda df: np.where(df['diff'] > 0.03, 1, 0))
 # Создаем признак, чтобы подсчитать время между заливками
 .assign(diff_time=dt.timedelta(minutes=45))
 )

# Подсчет правильного количества заливок
ice_df.loc[ice_df['flood'] == 1, 'diff_time'] = ice_df.loc[ice_df['flood'] == 1, 'time'].diff().fillna(dt.timedelta(minutes=100))
ice_df.loc[ice_df['diff_time'] < dt.timedelta(minutes=45), 'flood'] = 0

In [8]:
flood_df = ice_df.groupby(pd.to_datetime(ice_df['time'].dt.date))[['flood']].sum().reset_index()
data = data.merge(flood_df, on='time')

In [9]:
model = pickle.load(open('models/baseline_polynominal_compressors.sav', 'rb'))
xpoly = pickle.load(open('models/poly_features.sav', 'rb'))

In [10]:
xfeatures = xpoly.fit_transform(data[['temp_outside', 'flood']])
data['baseline'] = model.predict(xfeatures)
data['baseline'] = np.where(data['baseline'] > data['compressors'], data['baseline'], data['compressors'])
data['economy'] = data['baseline'] - data['compressors']

# <h2>Формирование витрины данных</h2>

In [11]:
# Подключаемся к БД с правами загрузки
HOST = os.getenv('SUDO_HOST')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('SUDO_PASSWORD')
CA = os.getenv('CA')
READONLY = os.getenv('SUDO_READONLY')

client = Client(host=HOST, user=USERNAME, password=PASSWORD, ca_certs=CA, secure=True)

In [12]:
# Базы данных 
client.execute('show databases')

[('akarmanov_test_db',)]

In [13]:
# Создаем базу данных объекта с таблицей витрины
client.execute('USE akarmanov_test_db')
client.execute('''
CREATE OR REPLACE TABLE baseline
(
    time Datetime,
    baseline Float32,
    compressors Float32,
    economy Float32,
    temp_outside Float32,
    flood UInt8
)
ENGINE = ReplacingMergeTree(time)
ORDER BY time
''')

[]

In [14]:
# Записть данных в БД
client.insert_dataframe(
    'INSERT INTO "baseline" (time, baseline, compressors, economy, temp_outside, flood) values',
    data[['time', 'baseline', 'compressors', 'economy', 'temp_outside', 'flood']],
    settings=dict(use_numpy=True)
    )

314

In [15]:
client.query_dataframe('select * from baseline final')

,time,baseline,compressors,economy,temp_outside,flood
0,2022-09-20,1989.128540,1989.128540,0.000000,11.164126,4
1,2022-09-21,2148.062744,2042.537231,105.525528,9.081195,13
2,2022-09-22,2020.460083,2001.760498,18.699591,10.449581,9
3,2022-09-23,2044.123169,2004.988525,39.134720,10.587442,10
4,2022-09-24,2059.048584,2059.048584,0.000000,7.949948,10
...,...,...,...,...,...,...
309,2023-07-26,1827.163086,1751.236938,75.926170,20.418806,10
310,2023-07-27,1836.449951,1836.449951,0.000000,20.129503,9
311,2023-07-28,1812.578979,1532.061768,280.517273,19.396645,7
312,2023-07-29,1889.282349,1282.816406,606.465942,15.569622,5
